In [1]:
from dotenv import dotenv_values
from requests import get
from requests.models import HTTPError
from bs4 import BeautifulSoup

import pandas as pd

In [2]:
CONFIG = dotenv_values('.env')

In [3]:
def requisicao_api_alura():
    try:
        response = get('https://cursos.alura.com.br/api/categorias')
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'Um erro HTTP ocorreu: {http_err}')
    except Exception as err:
        print(f'Um erro ocorreu: {err}')
    else:
        return response.json()

In [4]:
def subcategorias():
    dados = requisicao_api_alura()
    url = 'https://cursos.alura.com.br/forum/subcategoria-'
    urls_subcategorias = []
    nomes_categorias = []
    nomes_subcategorias = []
    for item in dados:
        for item_subcategoria in item['subcategorias']:
            nomes_subcategorias.append(item_subcategoria['nome'])
            nomes_categorias.append(item['nome'])
            urls_subcategorias.append(f'{url}{item_subcategoria["slug"]}/sem-resposta')
    df = pd.DataFrame({'Subcategoria': nomes_subcategorias, 'Categoria': nomes_categorias, 'Url': urls_subcategorias})
    return df

In [5]:
def requisicao(url):
    cookies = {'caelum.login.token':CONFIG['COOKIE']}
    try:
        response = get(url, cookies=cookies)
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'Um erro HTTP ocorreu: {http_err}')
    except Exception as err:
        print(f'Um erro ocorreu: {err}')
    else:
        soup = BeautifulSoup(response.text,'html.parser')
        return soup

In [18]:
def roda_scraping_subcategorias(df):
    topicos_sem_resposta = []
    for url in df['Url']:
        soup = requisicao(url)
        paginacao = soup.find('nav', {'class': 'busca-paginacao-links'})
        qtd_topicos = 0
        if paginacao:
            ultima_pagina = paginacao.get_text().split()[-1]
            qtd_topicos += (int(ultima_pagina)-1)*20
            url += f'/{ultima_pagina}'
            soup = requisicao(url)
        qtd_topicos += len(soup.find_all('div', {'class': {'forumList-item-subject'}}))
        topicos_sem_resposta.append(qtd_topicos)
    df['Tópicos sem resposta'] = topicos_sem_resposta
    return df

In [20]:
df = subcategorias()
roda_scraping_subcategorias(df)

,Subcategoria,Categoria,Url,Tópicos sem resposta
0,Lógica de programação,Programação,https://cursos.alura.com.br/forum/subcategoria...,33
1,Python,Programação,https://cursos.alura.com.br/forum/subcategoria...,57
2,Dart,Programação,https://cursos.alura.com.br/forum/subcategoria...,1
3,JavaScript,Programação,https://cursos.alura.com.br/forum/subcategoria...,4
4,Node.JS,Programação,https://cursos.alura.com.br/forum/subcategoria...,83
...,...,...,...,...
110,Vendas,Inovação & Gestão,https://cursos.alura.com.br/forum/subcategoria...,13
111,Práticas de Gestão - Waterfall & Híbridas,Inovação & Gestão,https://cursos.alura.com.br/forum/subcategoria...,3
112,Marketing Pessoal,Inovação & Gestão,https://cursos.alura.com.br/forum/subcategoria...,6
113,Conformidade Legal,Inovação & Gestão,https://cursos.alura.com.br/forum/subcategoria...,0
